#MNIST 예시

In [ ]:
import torch
# 신경망 구축에 필요한 여러 매서드를 담은 torch.nn
import torch.nn as nn
# torch.nn의 모든 함수를 포함, 손실/활성화/풀링/합성곱/선형 및 기타 신경망 함수 가 포함됨
import torch.nn.functional as F
# 최적화(신경망의 가중치,매개변수 조정을 위해 오차를 역전파 하는 과정) 모듈이 담김
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
class ConvNet(nn.Module):
  """
  kernel_size : 보통 홀수. 너무 작으면 픽셀을 처리하는 kernel이 이웃 픽셀의 정보를 가지지 못한다.너무 크면 이미지 내에서 정밀하지
                않은 특징을 얻게 됨. 작은 kernel_size의 많은 layer를 쓰면 네트워크가 깊어지고, 더 복잡한 특징 학습 가능.

  feature map : 이미지 데이터에서 픽셀 정보를 담고 있는 channel(차원)을 의미. 이미지에서 더 많은 특징을 추출하려거든 channel을 크게
                하면 된다.

  input shape이 28x28x1임

  Conv2d(input_channel, output_channel, kernel_size, stride)
  """
  # 각 layer의 뉴런개수 및 layer들 정의
  def __init__(self):
    super(ConvNet, self).__init__()

    # 합성곱 layer
    self.cn1 = nn.Conv2d(1,16,3,1)
    self.cn2 = nn.Conv2d(16,32,3,1)
    # 드롭아웃 layer
    self.dp1 = nn.Dropout(0.1)
    self.dp2 = nn.Dropout(0.25)
    # fully-connected(fc) layer, 4608=12x12x32
    self.fc1 = nn.Linear(4608, 64)
    # 최종 출력은 10개 클래스 중 하나
    self.fc2 = nn.Linear(64,10)

  def forward(self, x):
    x = self.cn1(x)
    x = F.relu(x)
    x = self.cn2(x)
    x = F.relu(x)
    # kernel size가 2x2
    x = F.max_pool2d(x,2)
    x = self.dp1(x)
    # 1차원 백터로 평면화
    x = torch.flatten(x,1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dp2(x)
    x = self.fc2(x)
    # 모델의 예측을 out에 담음
    out = F.log_softmax(x, dim=1)
    return out

In [ ]:
# 훈련 루틴 정의
def train(model, device, train_dataloader, optim, epoch):
  # 모델 훈련
  model.train()
  # batch 단위로 반복
  for b_i, (X,y) in enumerate(train_dataloader):
    # 주어진 로컬 메모리에 데이터셋 사본 만듦
    X,y = X.to(device), y.to(device)
    # 이전에 계산했던 gradient를 초기화(이전 값들은 이미 이전단계의 파라미터 수정할 때 쓰였으니까 노필요)
    optim.zero_grad()
    # 주어진 입력데이터를 활용하여 모델 예측 실행
    pred = model(X)
    # negative log liklihood, 모델예측값과 실제값 사이의 손실 계산
    loss = F.nll_loss(pred, y)
    # 역전파, 자동미분됨
    loss.backward()
    # 가중치 조정
    optim.step()
    if b_i % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, b_i * len(train_dataloader), len(train_dataloader.dataset),100. * b_i / len(train_dataloader), loss.item()))

In [ ]:
# 테스트 루틴 정의
def test(model, device, test_dataloader):
  """
  torch.no_grad : 추론(평가) 과정에서 사용하는 autograd 끄는 함수. 모델이 파라미터 업데이트 안 하고(즉, 학습X) 단순히 예측/추론만 함.

  평균적인 손실(오차) 정도를 구하기 위해 loss를 합한다.
  """

  # 모델 성능 평가
  model.eval()
  loss = 0
  success = 0
  with torch.no_grad():
    for X, y in test_dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      # 배치별 손실의 합(옵티마이저X->모델 가중치 조정X->모델평가를 위해 배치 단위로 오차 합함)
      loss += F.nll_loss(pred, y, reduction='sum').item()
      pred = pred.argmax(dim=1, keepdim=True)
      success += pred.eq(y.view_as(pred)).sum().item()
    loss /= len(test_dataloader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(loss, success, len(test_dataloader.dataset),100. * success / len(test_dataloader.dataset)))

In [ ]:
"""
torch.DataLoader : DataLoader에 입력되는 dataset을 모델에 배치입력하기 쉽게 만들어주는 모듈(추가적인 동작이 필요할 경우 DataSet을 상속받는
                   사용자 정의 DataSet을 만들어 그 DataSet을 DataLoader에 넣기도 함). 그래서 Dataset갹채에서 배치단위 데이터를 가져온 걸 반환.

transform 파리미터는 MNIST 이미지 데이터에 대한 전처리 과정을 정의.

transforms.Compose() 괄호 안에 들어가는 변환함수들을 담은 리스트는 순차적으로 입력데이터에 적용됨

transforms.Normalize((mean, std))

batch_size : 한번에 모델에 입력되는 데이터 묶음. 총 1000개의 이미지데이터가 있다면 batch_size가 32이라면 한 번 학습 시에
             32개의 이미지(데이터,혹은 샘플)가 쓰인다는 것. 그리고 이때 1000/32 약 32개의 배치(데이터묶음)가 생기는데
             마지막 배치의 경우 (1000-31)*32로 8개의 이미지데이터가 담김
# # 훈련 데이터 불러오기
# train_dataloader = torch.utils.data.DataLoader(datasets.MNIST("../data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor(),
#                                                                                                                                   transforms.ToTensor(),
#                                                                                                                                   transforms.Normalize((0.1302,),(0.3069))])),
#                                                batch_size=32, shuffle=True)
# # 성능 평가를 위한 데이터 불러오기
# test_dataloader = torch.utils.data.DataLoader(datasets.MNIST("../data", train=False, transform=transforms.Compose([transforms.ToTensor(),
#                                                                                                                    transforms.Normalize((0.1302,),(0.3069))])),
#                                                batch_size=500, shuffle=True)
"""

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_dataloader = torch.utils.data.DataLoader(dataset1,batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset2, batch_size=500, shuffle=True)

In [ ]:
# 무작위성X/재현가능성 위해 시드값 설정
torch.manual_seed(0)
# 연산 수행할 장치 지정
device = torch.device("cpu")
# 모델 객체 생성
model = ConvNet()
# 옵티마이저 객체 생성
optimizer = optim.Adadelta(model.parameters(), lr=0.5)

In [ ]:
# 모델을 실제로 훈련,테스트
# %capture
for epoch in range(1,3):
  train(model, device, train_dataloader, optimizer, epoch)
  test(model, device, test_dataloader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311855
Train Epoch: 1 [18750/60000 (1%)]	Loss: 1.873572
Train Epoch: 1 [37500/60000 (1%)]	Loss: 1.161529
Train Epoch: 1 [56250/60000 (2%)]	Loss: 0.849931
Train Epoch: 1 [75000/60000 (2%)]	Loss: 0.852791
Train Epoch: 1 [93750/60000 (3%)]	Loss: 0.439481
Train Epoch: 1 [112500/60000 (3%)]	Loss: 0.480759
Train Epoch: 1 [131250/60000 (4%)]	Loss: 0.413792
Train Epoch: 1 [150000/60000 (4%)]	Loss: 0.409818
Train Epoch: 1 [168750/60000 (5%)]	Loss: 0.342001
Train Epoch: 1 [187500/60000 (5%)]	Loss: 0.471768
Train Epoch: 1 [206250/60000 (6%)]	Loss: 0.284015
Train Epoch: 1 [225000/60000 (6%)]	Loss: 0.418780
Train Epoch: 1 [243750/60000 (7%)]	Loss: 0.566111
Train Epoch: 1 [262500/60000 (7%)]	Loss: 0.413485
Train Epoch: 1 [281250/60000 (8%)]	Loss: 0.603133
Train Epoch: 1 [300000/60000 (9%)]	Loss: 0.130813
Train Epoch: 1 [318750/60000 (9%)]	Loss: 0.292469
Train Epoch: 1 [337500/60000 (10%)]	Loss: 0.090156
Train Epoch: 1 [356250/60000 (10%)]	Loss: 0.190347
Train Ep

In [ ]:
# 모델을 훈련 후 테스트셋을 통해 성능도 검증했으니 샘플 이미지에서 추론이 맞는지 확인
test_samples = enumerate(test_dataloader)
b_i, (sample_data, sample_targets) = next(test_samples)

# 확률이 가장 높은 클래스(max)를 선택. model(sample_data)[1]은 숫자 분류 결과 배열.
print(f"model prediction : {model(sample_data).data.max(1)[1][0]}")
print(f"Ground truth : {sample_targets[0]}")

model prediction : 0
Ground truth : 0


#CNN과 LSTM 결합하기

이 챕터에서는 CNN과 LSTM을 결합한 하이브리드 모델을 이용한 이미지 캡션(이미지 설명) 실습 수행.

CNN은 이미지를 가져와 고차원 특징이나 임베딩을 출력하는 인코더로 사용.
LSTM은 CNN의 마지막 은닉층을 입력값으로 받아 텍스트를 생성하는 디코더.

LSTM은 t=0에서 CNN의 임베딩(벡터)을 입력으로 가져옴.

In [ ]:
## linux
!apt-get install wget

## create a data directory
!mkdir data_dir

## download images and annotations to the data directory
!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip -P ./data_dir/
!wget http://images.cocodataset.org/zips/train2014.zip -P ./data_dir/
!wget http://images.cocodataset.org/zips/val2014.zip -P ./data_dir/

## extract zipped images and annotations and remove the zip files
!unzip ./data_dir/captions_train-val2014.zip -d ./data_dir/
!rm ./data_dir/captions_train-val2014.zip
!unzip ./data_dir/train2014.zip -d ./data_dir/
!rm ./data_dir/train2014.zip
!unzip ./data_dir/val2014.zip -d ./data_dir/
!rm ./data_dir/val2014.zip

In [7]:
import os
# 자연어 툴킷, 사전을 구축할 때 사용
import nltk
# 파이썬 객체를 직렬화/역직렬화 하는 라이브러리
import pickle
import numpy as np
# 이미지 파일을 다룰 수 있게 해주는 라이브러리
from PIL import Image
# 데이터의 빈도수를 셀 수 있는 도구
from collections import Counter
# COCO데이터셋을 가지고 작업할 때 유용
from pycocotools.coco import COCO
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision.models as model
import torchvision.transforms as transform
import matplotlib.pyplot as plt
# 다양한 길이의 문장에 패딩을 적용해 고장된 길이의 문장으로 변환할 때 사용하는 라이브러리
from torch.nn.utils.rnn import pack_padded_sequence

In [3]:
# punkt 토크나이저 모델 다운. 텍스트를 단어로 토큰화
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
class Vocab(object):
    """
    간단한 사전 객체

    vocabulary(사전) : 단어집합 OR 자연어 처리 시스템이 인식,처리 할 수 있는 단어,토큰 집합. 즉,
                       단어를 숫자로 매핑하여 그걸 저장하고 있음.

    wrapper : vocabulary을 관리하는 클래스/함수.
    """
    def __init__(self):
        # word에서 index로 매핑된 값을 저장하는 딕셔너리. ex) 'hello':1 이런 걸 저장
        self.w2i = {}
        # index에서 단어로의 매핑을 저장하는 딕셔너리. ex) 1:'hello' 이런 걸 저장
        self.i2w = {}
        # 새로운 단어가 추가될 때마다 값이 증가
        self.index = 0

    def __call__(self, token):
        # 만약 아무 단어도 매핑이 안 되었으면
        if not token in self.w2i:
            # unknown toekn의 인덱스를 반환
            return self.w2i['<unk>']
        # 만약 토큰이 존재하면(즉 단어가 이미 매핑 되어 있었다면), 해당 단어에 대응하는 인덱스 출력
        return self.w2i[token]

    # 단어집합의 크기(단어 개수)를 반환
    def __len__(self):
        return len(self.w2i)

    # 새로운 단어를 사전에 추가
    def add_token(self, token):
        # 단어가 매핑되어 저장되어 있지 않았다면
        if not token in self.w2i:
            # 해당 단어(토큰)가 key가 되고, index를 value로 설정하여 딕셔너리에 저장
            self.w2i[token] = self.index
            # 얘도 마찬가지
            self.i2w[self.index] = token
            # 단어 새로 추가됐으니까 또다른 새로운 단어의 value로 쓰일 인덱스 값 만들어주기
            self.index += 1


# 텍스트 토큰을 숫자 토큰으로 전환할 수 있는 사전을 구축하는 함수
def build_vocabulary(json, threshold):
  # 사전 wrapper를 구축(사전(단어집합)을 관리하는 함수를 구축)
  coco = COCO(json)
  counter = Counter()
  # COCO객체인 coco의 ann(annotation)의 키값들을 ids에 담기(ann이 dict로 추정)
  ids = coco.anns.keys()
  for i, id in enumerate(ids):
    # 캡션(이미지 설명 문장)을 각 이미지로부터 가져오기
    caption = str(coco.anns[id]['caption'])
    # tokenize 모듈에 포함된 word_tokenize 함수로 문자열을 단어 단위로 분리(공백, 구두점 등도 알아서 처리)
    tokens = nltk.tokenize.word_tokenize(caption.lower()) # caption을 소문자로 변환, 리스트가 반환됨
    # iterable을 인자로 받아 그 안의 요소들의 빈도를 Counter 객체에 추가(OR 기존을 업데이트)
    counter.update(tokens)
    if (i+1) % 1000 == 0:
      print("[{}/{}] Tokenized the captions.".format(i+1, len(ids)))
  # 빈도수가 threshold이상인 토큰들만 분류하여 리스트에 담기
  tokens = [token for token, cnt in counter.items() if cnt>=threshold]
  # vocab wrapper를 만들고 특수한 토큰을 수동으로 추가
  vocab = Vocab()
  vocab.add_token("<pad>")
  vocab.add_token("<start>")
  vocab.add_token("<end>")
  vocab.add_token("<unk>")
  # w2i, i2w에 tokenize 후 넣어주기(인덱스는 add 되면서 자동으로 각 token에 할당된다)
  for _, token in enumerate(tokens):
    vocab.add_token(token)
  # tokenize 후 indexing도 다 된(즉, 매핑이 끝난) 사전을 반환
  return vocab

In [ ]:
# 4번 이상 나온 단어들만 골라내어 하나의 집합으로 만듦
vocab = build_vocabulary(json='data_dir/annotations/captions_train2014.json', threshold=4) # 책 따라하니까 설명이 생략된 게 좀 있어 보이는데 COCO 웹사이트 가서 captions 직접 다운함
vocab_path = './data_dir/vocabulary.pkl'
with open(vocab_path, 'wb') as f: # ./data_dir vocabulary.pkl 파일을 만들어 바이너리 쓰기 모드로 열어서
    pickle.dump(vocab, f) # 사전객체(vocab)를 직렬화하여 파일에 저장->사전 객체를 로컬에 저장하여 나중에 사전 구축 함수를 재구성하는 수고를 덜 수 있음.
print("Total vocabulary size: {}".format(len(vocab)))
print("Saved the vocabulary wrapper to '{}'".format(vocab_path))

In [10]:
# CNN모델의 첫 번째 계층에 입력으로 제공될 수 있도록 전체 이미지를 고정된 모양으로 변형

# 개별 이미지의 사이즈를 정해진 shape으로 바꾸기
def reshape_image(image, shape):
  return image.resize(shape, Image.ANTIALIAS)

# 이미지 데이터들을 리사이징
def reshape_images(image_path, output_path, shape):
  """
  image_path : shape을 resize할 이미지들이 모여있는 위치

  output_path : 변형을 마친 이미지들이 저장될 위치
  """

  # output_path라는 폴더가 없다면
  if not os.path.exists(output_path):
    # 하나 만들어라
    os.makedirs(output_path)
  # image_path 디렉토리에 있는 파일과 폴더를 가져옴
  images = os.listdir(image_path)
  num_img = len(images)
  # images 변수에 저장된 개별 image들을
  for i, img in enumerate(images):
    # image_path에 있는 img 파일들을 open하고(이미지 파일은 바이너리니까 b) f에 데이터를 담고
    with open(os.path.join(image_path, img), 'r+b') as f:
      # PIL 라이브러리의 Image메서드를 통해 해당 f에 담긴 이미지 파일 객체를 열어 그걸 변수 image에 담고
      with Image.open(f) as image:
        # 모든 이미지들이 사이즈가 제각각이므로 이미지를 고정된 shape으로 다 reshaping
        image = reshape_image(image, shape)
        # Pillow의 Image 객체의 save메서드를 이용해 output_path 경로에 img라는 이름의 이미지 파일 저장
        image.save(os.path.join(output_path, img), image.format) # 그리고 이미지 파일 객체의 파일 형식 해당 이미지 파일의 현재 형식으로 설정
      if (i+1) % 100 == 0:
        print("[{}/{}] Resized the images and saved into {}.".format(i+1, num_img, output_path))

In [11]:
image_path = './data_dir/train2014/'
output_path = './data_dir/resized_images/'
# 모든 이미지를 256X256의 size(=shape)로 만든다
image_shape = [256, 256]
reshape_images(image_path, output_path, image_shape)

<ipython-input-10-ceba774c0ee7>:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  return image.resize(shape, Image.ANTIALIAS)


[100/82783] Resized the images and saved into ./data_dir/resized_images/.
[200/82783] Resized the images and saved into ./data_dir/resized_images/.
[300/82783] Resized the images and saved into ./data_dir/resized_images/.
[400/82783] Resized the images and saved into ./data_dir/resized_images/.
[500/82783] Resized the images and saved into ./data_dir/resized_images/.
[600/82783] Resized the images and saved into ./data_dir/resized_images/.
[700/82783] Resized the images and saved into ./data_dir/resized_images/.
[800/82783] Resized the images and saved into ./data_dir/resized_images/.
[900/82783] Resized the images and saved into ./data_dir/resized_images/.
[1000/82783] Resized the images and saved into ./data_dir/resized_images/.
[1100/82783] Resized the images and saved into ./data_dir/resized_images/.
[1200/82783] Resized the images and saved into ./data_dir/resized_images/.
[1300/82783] Resized the images and saved into ./data_dir/resized_images/.
[1400/82783] Resized the images an

지금까지 이미지와 해당 이미지의 캡션 데이터를 받아 전처리를 했다(reshaping, tokenizing) 이제 이 데이터를 파이토치 데이터셋으로 변환

In [15]:
class CustomCocoDataset(data.Dataset):
  def __init__(self, data_path, coco_json_path, vocabulary, transform=None):
    """
    data_path : 이미지 데이터 경로
    coco_json_path : coco annotation 파일 경로
    vocabulary : 우리가 만든 사전객체(vocabulary.pkl)
    transform : 이미지 transformer(이미 resizing했으니까 노필요)
    """
    self.root = data_path
    self.coco_data = COCO(coco_json_path) # 주어진 coco_json_path(annotation파일의 경로)로부터 JSON파일을 읽어 데이터 제공
    self.indices = list(self.coco_data.anns.keys())
    self.vocabulary = vocabulary
    self.trasnform = transform

  # image와 caption의 한 페어를 반환하는 메서드
  def __getitem__(self, idx):
    coco_data = self.coco_data
    vocabulary = self.vocabulary
    annotation_id = self.indices(idx)
    caption = coco_data.anns[annotation_id]['caption']
    image_id = coco_data.anns[annotation_id]['image_id']
    image_path = coco_data.loadImgs(image_id)[0]['file_name']
    image = Image.open(os.path.join(self.root, image_path)).convert("RGB")
    # 만약 transform을 위한 모델을 따로 넣었다면
    if self.transform is not None:
      # 그 변환 모델로 이미지 변형
      image = self.transform(image)
    # caption들을 tokenize(단어->숫자화)
    word_tokens = nltk.tokenize.word_tokenize(str(caption).lower)
    caption = []
    caption.append(vocabulary('<start>'))
    # 토큰화된 단어들을 하나의 리스트에 모아서 한 번에 caption리스트에 넣기
    caption.extend([vocabulary(token) for token in word_tokens])
    caption.append(vocabulary('<end>'))
    # true값들을 모은 리스트를 파이토치 텐서로 바꾸는 것.
    ground_truth = torch.Tensor(caption)

  def __len__(self):
    return len(self.indicies)


# X, y 형태로 데이터의 미니배치를 반환하는 함수
def collate_function(data_batch):
  """
  (image, caption)형태의 튜플들을 담은 리스트로부터 미니배치 텐서를 만들어주는 함수

  기본적으로 collate_fn가 제공되지만, 캡션(자연어)에 대한 패딩과 병합이 되지 않으므로 custom으로 만드는 것

  반환하는 것은 (batch_size, 3, 256, 256) shape의 images 텐서, (batch_size, padded_length) shape의 targets 텐서,
  padding 된 캡션의 길이
  """

  # (image, caption)이 들어있는 data_batch 리스트를 내림차순으로 정렬하는데 기준(key)은 각각의 (image, caption)의 caption길이
  data_batch.sort(key=lambda d: len(d[1]), reverse=True)
  imgs, caps = zip(*data_batch) # 각 (image, caption) 쌍을 (img1, img2...), (cap1, cap2...)의 형태로 각 변수에 할당
  # imgs에는 batch_size만큼의 img들이 들어 있고, 각 img들은 (3,256,256)의 shape의 matrix인데
  imgs = torch.stack(imgs, 0) # stack 통해서 (batch_size,3,256,256)의 shape으로 변함

  # 캡션도 원래는 1차원이지만, batch_size를 고려하여 (batch_size, 캡션 길이)의 2차원으로 변경
  cap_lens = [len(cap) for cap in caps]
  # 모든 캡션을 담을 수 있는 빈 텐서 생성
  tgts = torch.zeros(len(caps), max(cap_lens)).long() # len(caps): 배치 내의 캡션 개수, 즉 배치 크기 max(cap_lens): 배치에 가장 긴 캡션. 얘를 기준으로 모든 캡션을 동일 길이로 맞추기 위한 텐서의 두 번째 차원
  for i, cap in enumerate(caps):
    end = cap_lens[i] # 각 캡션의 실제 길이(단어 수)를 가져와서
    tgts[i, :end] = cap[:end] # i번째 캡션에 대해 텐서를 채워 넣는다
  return imgs, tgts, cap_lens


# coco dataset을 위한 torch.utils.data.DataLoader을 반환하는 함수. 훈련모드에서 데이터의 미니 배치를 가져오는데 유용
def get_loader(data_path, coco_json_path, vocabulary, transform, batch_size, shuffle, num_workers):
  """
  반환하는 것은 (batch_size, 3, 256, 256) shape의 images 텐서, (batch_size, padded_length) shape의 targets 텐서,
  padding 된 캡션의 길이
  """
  coco_dataset = CustomCocoDataset(data_path=data_path, coco_json_path = coco_json_path, vocabulary=vocabulary, transform=transform)

  # 한 번의 주기(iteration)에서 (images, captions, lengths)를 반환
  custom_data_loader = torch.utils.data.DataLoader(dataset=coco_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate_function)
  return custom_data_loader

In [ ]:
class CNNModel(nn.Module):
  """
  embedding_layer :

  lstm_layer :
  """

  def __init__(self, embedding_size):
    # 사전학습 된 ResNet-152를 불러온다 CNN의 출력 결과를 생성하는 fc layer를 대체

class LSTMModel(nn.Module):
  def __init__(self,embedding_size, hidden_size, vocabulary_size, num_layers, max_seq_len=20):
    super(LSTMModel, self).__init__()
    self.embedding_layer = nn.Embedding(vocabulary_size, embedding_size)
    self.lstm_layer = nn.LSTM(embedding_size, hidden_layer_size, num_layers, batch_first=True)
    self.linear_layer = nn.Linear(hidden_layer_size, vocabulary_size)
    self.max_seq_len = max_seq_len

  def forward(self, input_features, caps, lens):
    """
    이미지 특징 벡터를 decode 하고 해당 이미지의 caption을 만들기(예측하기)

    embeddings : 단어를 고정 길이의 실수 벡터로 변환한 것.

    두번째 embeddings : input_features.unsqueeze(1)를 통해 input_feature의 두 번째 차원에 새로운 축을 추가하여 이미지의 특징 벡터가 (batch_size,1,feature_size)
                        가 될 수 있게 함. 왜 이렇게 함?
                        -> 캡션의 각 단어의 shape이 (batch_size, caption_length, embedding_size)인데 input_feature의 경우 (batch_size, feature_size)라 차원이
                           안 맞아서.


    """

    # caps에 있는 단어 인덱스들을 임베딩 벡터로 변환
    embeddings = self.embedding_layer(caps)
    embeddings = torch.cat((input_features.unsqueeze(1), embeddings), 1)
    lstm_input = pack_padded_sequence(embeddings, lens, batch_first=True)
    hidden_variables, _ = self.lstm_layer(lstm_input)
    model_outputs = self.linear_layer(hidden_variables[0])
    return model_outputs